# <center> Практические задания по цифровой обработке сигналов </center>
# <center> Специальная лабораторная работа № 2 </center>
# <center> ДЕТЕКТИРОВАНИЕ ПОЛА ПО РЕЧЕВОЙ ЗВУКОЗАПИСИ </center>

In [2]:
import os
import pandas as pd
import torchaudio
import torch
import numpy as np
from sklearn.mixture import GaussianMixture
from torchdata.datapipes.iter import FileLister


Получение всех идентификаторов дикторов, которые находятся в наборе аудиоданных vox1_test_wav.zip

In [3]:
speaker_id_list = os.listdir("wav/")
print(speaker_id_list)

['id10270', 'id10271', 'id10272', 'id10273', 'id10274', 'id10275', 'id10276', 'id10277', 'id10278', 'id10279', 'id10280', 'id10281', 'id10282', 'id10283', 'id10284', 'id10285', 'id10286', 'id10287', 'id10288', 'id10289', 'id10290', 'id10291', 'id10292', 'id10293', 'id10294', 'id10295', 'id10296', 'id10297', 'id10298', 'id10299', 'id10300', 'id10301', 'id10302', 'id10303', 'id10304', 'id10305', 'id10306', 'id10307', 'id10308', 'id10309']


Загрузка метаинформации по отношению к дикторам из файла vox1_meta.csv

In [4]:
all_speaker_info = pd.read_csv("vox1_meta.csv", delimiter="\t")
all_speaker_info.columns = ["id","name","gender", "nationality", "set"]
print(all_speaker_info.head())

        id          name gender nationality  set
0  id10001  A.J._Buckley      m     Ireland  dev
1  id10002   A.R._Rahman      m       India  dev
2  id10003    Aamir_Khan      m       India  dev
3  id10004   Aaron_Tveit      m         USA  dev
4  id10005     Aaron_Yoo      m         USA  dev


Фильтрация метаданных дикторов, содержащихся в наборе данных

In [5]:
ds_speaker_info = all_speaker_info[all_speaker_info["id"].isin(speaker_id_list)]
print(ds_speaker_info.shape)

(40, 5)


Получить список из 15 случайных идентификаторов дикторов для каждого пола

In [6]:
male_id = ds_speaker_info[ds_speaker_info["gender"] == "m"]["id"]
male_id = male_id.sample(frac=1).reset_index(drop=True).iloc[:15]

female_id = ds_speaker_info[ds_speaker_info["gender"] == "f"]["id"]
female_id = female_id.sample(frac=1).reset_index(drop=True).iloc[:15]

print(male_id)

0     id10295
1     id10277
2     id10278
3     id10294
4     id10292
5     id10275
6     id10293
7     id10301
8     id10276
9     id10271
10    id10272
11    id10279
12    id10309
13    id10304
14    id10273
15    id10297
16    id10302
17    id10305
18    id10298
19    id10290
20    id10284
21    id10274
22    id10296
23    id10283
24    id10300
Name: id, dtype: object


Вычислить  наборы  из  23 мел-частотных кепстральных коэффициентов для аудиоданных

In [7]:
def calc_mfcc(id_list):
    for id in id_list:
        file_list = list(FileLister(root=f"wav\\{id}", recursive=True))
        wav_list = [i for i in file_list if i.endswith(".wav")]
        for file_path in wav_list:
            path_to_mfcc = file_path.split(".")[0]+".pt"
            if not os.path.isfile(path_to_mfcc):
                # загрузить звукозапись
                waveform, sample_rate = torchaudio.load(file_path)

                # вычислить mfcc для записи
                mfcc = torchaudio.compliance.kaldi.mfcc(waveform, sample_frequency=sample_rate, num_ceps=23)
                
                # сохранить коэффициенты на диск
                torch.save(mfcc, path_to_mfcc)

In [8]:
calc_mfcc(male_id)
calc_mfcc(female_id)

Разделить данные на тренировочное и тестовое

In [9]:
male_train_id = male_id.iloc[:10]
male_test_id = male_id.iloc[10:15]

female_train_id = female_id.iloc[:10]
female_test_id = female_id.iloc[10:15]

Загрузить сохраненные наборы коэффициентов с диска

In [10]:
def load_mfcc(id_list, type):
    list_mfcc = []
    for id in id_list:
        file_list = list(FileLister(root=f"wav\\{id}", recursive=True))
        mfcc_path_list = [i for i in file_list if i.endswith(".pt")]
        for path in mfcc_path_list:
            list_mfcc.append(torch.load(path))
        
    if type == "test":
        return [i.numpy() for i in list_mfcc]
    elif type == "train":
        return torch.cat(list_mfcc).numpy()

In [11]:
X_male_train = load_mfcc(male_train_id, "train")
X_female_train = load_mfcc(female_train_id, "train")

In [12]:
y_male = load_mfcc(male_test_id, type="test")
y_female = load_mfcc(female_test_id, type="test")

Обучить две многомерные модели гауссовой смеси, состоящих  из 5 гауссоид

In [13]:
male_model = GaussianMixture(n_components=5, random_state=0).fit(X_male_train)
female_model = GaussianMixture(n_components=5, random_state=0).fit(X_female_train)

In [ ]:
def predict(y):
  res = np.array([])
  for i in y:
    res = np.append(res, np.sum(male_model.score_samples(i))-np.sum(female_model.score_samples(i)))
  return res
  
res_male = predict(y_male)
res_female = predict(y_female)

Точность для дикторов мужского пола и дикторов женского пола

In [ ]:
accuracy_male = np.sum(np.array(res_male) > 0, axis=0)/len(res_male)
accuracy_female = np.sum(np.array(res_female) < 0, axis=0)/len(res_female)

print("Accuracy for male gender: {:10.2f}%".format(100*accuracy_male))
print("Accuracy for female gender: {:10.2f}%".format(100*accuracy_female))

Accuracy for male gender:      98.88%
Accuracy for female gender:     100.00%
